In [1]:
import pandas as pd
import numpy as np

In [2]:
#보험 청구 금액 예측하는 인공지능 모델

In [3]:
#1. 데이터 준비
df=pd.read_csv('data/healthcare.csv',index_col=0)

In [4]:
df.head(5)

,Age,Gender,BMI,Region,Smoker,NumVisits,InsuranceClaim
0,51.0,Female,NaN,South,No,19.0,70.081987
1,92.0,Female,38.074006,West,Yes,13.0,92.747518
2,14.0,Male,NaN,North,No,8.0,46.794138
3,NaN,Female,27.020924,West,No,7.0,44.789132
4,60.0,Male,37.961368,North,No,16.0,71.790344


In [5]:
#데이터 전처리
#결측치 확인
df.isnull().sum()
#이번 프로젝트에서는 결측치를 드랍하는 대신 평균값으로 채울것이다.

Age               30
Gender             0
BMI               50
Region             0
Smoker             0
NumVisits         20
InsuranceClaim     0
dtype: int64

In [6]:
#타겟변수(y) 피처(X) 분리 
y=df['InsuranceClaim']
X=df.drop(columns='InsuranceClaim')

In [7]:
#2. 카테고리컬(범주형) 데이터와 수치형 데이터 처리
# 컬럼 이름을 분리해서 저장 (수치형, 범주형)

In [8]:
df.columns

Index(['Age', 'Gender', 'BMI', 'Region', 'Smoker', 'NumVisits',
       'InsuranceClaim'],
      dtype='object')

In [9]:
#수치형 컬럼 이름
numerical=['Age','BMI','NumVisits']

In [10]:
#범주형 컬럼 이름
categorical=['Gender','Region','Smoker']

In [11]:
#수치형 데이터의 결측치를 평균값으로 채우기
#범주형 데이터는 레이블인코딩, 원핫인코딩 한다.
#sklearn의 pipeline을 사용하면 간단하게 전처리를 할 수 있다.

In [12]:
#2.1 수치형 데이터 처리
#fillna() : 결측치를 채우는 함수 대신 SimpleImputer를 사용한다.


In [13]:
from sklearn.impute import SimpleImputer


imputer=SimpleImputer(strategy='mean') #디폴트가 mean이라 쓰지 않아도된다.

In [14]:
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer

In [15]:
label_encoder=OrdinalEncoder()
onehot_encoder=OneHotEncoder()

In [16]:
df['Gender'].nunique()

2

In [17]:
df['Region'].nunique()

4

In [18]:
df['Smoker'].nunique()

2

In [19]:
label_columns=['Gender','Smoker']
onehot_columns=['Region']

In [20]:
numerical_columns=['Age','BMI','NumVisits']

In [21]:
#각 전처리할 라이브러리 준비, 해당 라이브러리에 적용할 컬럼리스트를 준비했다.
#ColumnTransformer를 사용해서 각 라이브러리에 적용할 컬럼리스트를 전달한다.

In [22]:
preprocessor=ColumnTransformer([
    ('unm',imputer,numerical_columns),
    ('label',label_encoder,label_columns),
    ('onehot',onehot_encoder,onehot_columns)
])

In [34]:
#3. 파이프라인 생성
from sklearn.ensemble import RandomForestRegressor

In [36]:
regressor = RandomForestRegressor(random_state=42)

In [37]:
from sklearn.pipeline import Pipeline

In [38]:
pipeline=Pipeline(steps=[('preprocessing',preprocessor),('modeling',regressor)])
#~하고 ~하고 ~하고 이런식으로 파이프라인을 만들어준다.

In [39]:
pipeline

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('unm', SimpleImputer(),
                                                  ['Age', 'BMI', 'NumVisits']),
                                                 ('label', OrdinalEncoder(),
                                                  ['Gender', 'Smoker']),
                                                 ('onehot', OneHotEncoder(),
                                                  ['Region'])])),
                ('modeling', RandomForestRegressor(random_state=42))])

In [40]:
#4. 학습 데이터와 테스트 데이터 분리

In [41]:
from sklearn.model_selection import train_test_split

In [42]:
X_train,X_test,y_train,y_test=train_test_split(X,y,random_state=42,test_size=0.2)


In [43]:
pipeline.fit(X_train,y_train)

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('unm', SimpleImputer(),
                                                  ['Age', 'BMI', 'NumVisits']),
                                                 ('label', OrdinalEncoder(),
                                                  ['Gender', 'Smoker']),
                                                 ('onehot', OneHotEncoder(),
                                                  ['Region'])])),
                ('modeling', RandomForestRegressor(random_state=42))])

In [33]:
# 파이프라인 이용하면 좋은점
# 이 파이프라인만 저장하면 서비스배포할때 예측할 데이터만 그냥 넣어주면 파이프라인이 알아서 전처리하고 예측해준다.

In [44]:
#5. 모델 평가

In [45]:
from sklearn.metrics import mean_squared_error, r2_score

In [46]:
y_pred = pipeline.predict(X_test)

In [47]:
mean_squared_error(y_test, y_pred)

46.980189783960185

In [48]:
r2_score(y_test, y_pred)

0.7521137711976167

In [52]:
import joblib

In [54]:
joblib.dump(pipeline,'model/insurance_pipeline.pkl')

['model/insurance_pipeline.pkl']